In [51]:
# train3에서 deeplearning(pytorch)


In [52]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [53]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tqdm

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.utils.data as data
from sklearn.datasets import make_classification

In [54]:
import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

In [55]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [56]:
# 가상 데이터 생성
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
input_dim = 20
hidden_dim1 = 64
hidden_dim2 = 32
output_dim = 1

In [58]:
CFG = {
    'EPOCHS': 100,
    'LEARNING_RATE': 1e-3,
    'BATCH_SIZE': 32,
    'SEED': 42
}

In [59]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train = train.drop(['user_id'], axis=1)
test_id = test['user_id'].values
test = test.drop(['user_id'], axis=1)

In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  object 
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [61]:
# preferred_difficulty_level 열 인코딩
train.loc[train['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
train.loc[train['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
train.loc[train['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2

test.loc[test['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
test.loc[test['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
test.loc[test['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2
train['preferred_difficulty_level'] = train['preferred_difficulty_level'].astype(int)

In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [63]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()
categoric_col =['subscription_type']
label_encoders = {}
for col in categoric_col:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].fit_transform(test[col])

In [64]:
# 'payment_pattern' 열에 대한 고유값 확인
unique_values = train['payment_pattern'].unique()

# 각 고유값에 대해 새로운 열 생성 및 0 또는 1로 설정
for value in unique_values:
    train[f'payment_pattern_{value}'] = (train['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
train.drop('payment_pattern', axis=1, inplace=True)

for value in unique_values:
    test[f'payment_pattern_{value}'] = (test['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
test.drop('payment_pattern', axis=1, inplace=True)

In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  int32  
 11  customer_inquiry_history           10000 non-null  int6

In [66]:
train_x = train.drop('target', axis = 1)
train_y = train['target']
test_x = test

In [67]:
ros = RandomOverSampler(random_state=42)
train_x, train_y = ros.fit_resample(train_x, train_y)

In [68]:
train_x

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern_5,payment_pattern_6,payment_pattern_7,payment_pattern_0,payment_pattern_1,payment_pattern_3,payment_pattern_4,payment_pattern_2
0,13,14,14.946163,8.427187,18,16,68.360455,3,4,0,0,4,1,0,0,0,0,0,0,0
1,16,18,18.453224,72.646087,16,13,97.567322,2,3,1,0,1,0,1,0,0,0,0,0,0
2,22,1,16.195228,21.774492,13,14,94.358763,3,4,1,1,0,0,0,1,0,0,0,0,0
3,1,19,17.628656,42.659066,19,18,70.153228,0,3,0,0,1,0,0,0,1,0,0,0,0
4,4,5,21.390656,30.744287,19,10,81.917908,2,4,1,0,3,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12393,20,2,16.757896,124.875916,5,10,87.939213,1,5,0,0,0,0,0,0,1,0,0,0,0
12394,13,26,13.202407,0.093898,21,18,72.592572,5,5,0,1,3,0,0,0,0,0,1,0,0
12395,22,5,17.941786,12.046055,17,11,78.796829,2,2,0,0,2,1,0,0,0,0,0,0,0
12396,10,13,17.234217,2.674188,4,11,65.069547,2,3,0,0,0,1,0,0,0,0,0,0,0


In [69]:
train_y

0        0
1        1
2        1
3        1
4        1
        ..
12393    0
12394    0
12395    0
12396    0
12397    0
Name: target, Length: 12398, dtype: int64

In [70]:
train_x.fillna(0, inplace=True)
test_x.fillna(0, inplace=True)

In [71]:
# X와 Y로 나누기
X_train, X_test, y_train, y_test = train_test_split(train_x,train_y, test_size=0.2, random_state=40)

In [72]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [73]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        return torch.FloatTensor(self.X[index]), torch.LongTensor([self.Y[index]])

    def __len__(self):
        return len(self.X)

In [74]:
# 데이터 로드
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CustomDataset(X_test, y_test)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [75]:
# 모델 정의
class SimpleDNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(SimpleDNN, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [87]:
# 모델 초기화 및 손실 함수, 옵티마이저 설정
input_dim = 20  # 예시로 20개의 특성
hidden_dim1 = 64
hidden_dim2 = 32
output_dim = 2  # 이진 분류를 위해 출력 차원은 2로 설정
model = SimpleDNN(input_dim, hidden_dim1, hidden_dim2, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

SimpleDNN(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)

In [88]:
y_train

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [103]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch_inputs, batch_labels in train_loader:
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels.squeeze())
        loss.backward()
        optimizer.step()

    # 검증
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_labels.squeeze())

            _, preds = torch.max(val_outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())
        f1 = f1_score(all_labels, all_preds, average='macro')


    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, F1 Score (macro): {f1:.4f}')

Epoch [1/10], Loss: 0.7119, F1 Score (macro): 0.4482
Epoch [2/10], Loss: 0.7004, F1 Score (macro): 0.5237
Epoch [3/10], Loss: 0.7153, F1 Score (macro): 0.5055
Epoch [4/10], Loss: 0.6411, F1 Score (macro): 0.5137
Epoch [5/10], Loss: 0.6825, F1 Score (macro): 0.4926
Epoch [6/10], Loss: 0.6577, F1 Score (macro): 0.4752
Epoch [7/10], Loss: 0.6595, F1 Score (macro): 0.4705
Epoch [8/10], Loss: 0.6899, F1 Score (macro): 0.5241
Epoch [9/10], Loss: 0.6549, F1 Score (macro): 0.5287
Epoch [10/10], Loss: 0.6954, F1 Score (macro): 0.5221


In [104]:
correct = 0
total = 0

# 모델을 평가 모드로 설정
model.eval()

# 모델을 현재 사용 가능한 디바이스로 이동
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

with torch.no_grad():
    for inputs, labels in val_loader:
        # 데이터를 현재 사용 가능한 디바이스로 이동
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        # 이진 분류일 경우
        if outputs.size(1) == 1:
            predicted = (outputs.round()).int()
        # 다중 분류일 경우
        else:
            predicted = torch.argmax(outputs, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total

print("Validation Accuracy: {:.2%}".format(accuracy))

Validation Accuracy: 3199.27%


In [30]:
for batch_inputs, batch_labels in train_loader:
    # batch_inputs과 batch_labels은 첫 번째 배치의 데이터입니다.
    # 여기에서 원하는 확인 작업을 수행하십시오.
    print("Batch Inputs:", batch_inputs)
    print("Batch Labels:", batch_labels)

    # 원하는 확인 작업을 추가하십시오.

    break  # 첫 번째 배치만 확인하고 루프를 종료합니다.

Batch Inputs: tensor([[ 4.0000, 20.0000, 14.5183,  ...,  0.0000,  0.0000,  0.0000],
        [15.0000,  9.0000, 14.1491,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.0000, 27.0000, 11.8826,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 8.0000,  7.0000, 14.9663,  ...,  0.0000,  0.0000,  0.0000],
        [23.0000, 11.0000, 15.6848,  ...,  1.0000,  0.0000,  0.0000],
        [20.0000, 25.0000, 12.9032,  ...,  0.0000,  0.0000,  0.0000]])
Batch Labels: tensor([[0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
     

In [79]:
class SimpleDNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(SimpleDNN, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device))
    


In [80]:
model = SimpleDNN(input_dim, hidden_dim1, hidden_dim2, output_dim)

In [81]:
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # 예시로 CrossEntropyLoss 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [82]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [83]:
model.to(device)

SimpleDNN(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)

In [84]:
epochs = 10  # 적절한 에폭 설정
for epoch in range(epochs):
    model.train()
    for batch_inputs, batch_labels in train_loader:
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

    # 검증
    model.eval()
    with torch.no_grad():
        for val_batch_inputs, val_batch_labels in val_loader:
            val_batch_inputs, val_batch_labels = val_batch_inputs.to(device), val_batch_labels.to(device)

            val_outputs = model(val_batch_inputs)
            val_loss = criterion(val_outputs, val_batch_labels)

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [48]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [47]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())
    return np.mean(val_loss)

In [48]:
model = SimpleDNN(input_dim, hidden_dim1, hidden_dim2, output_dim)

In [49]:
class SimpleDNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims=[64, 32], dropout_rate=0.2):
        super(SimpleDNN, self).__init__()

        # 첫 번째 은닉층
        self.fc1 = nn.Linear(input_dim, hidden_dims[0])
        self.dropout1 = nn.Dropout(dropout_rate)

        # 두 번째 은닉층
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.dropout2 = nn.Dropout(dropout_rate)

        # 출력층
        self.fc3 = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        # 순전파
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        x = self.fc3(x)
        return F.softmax(x, dim=1)


In [27]:
model = SimpleDNN(input_dim=20, output_dim=1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [45]:
sample_submission = pd.read_csv('../data/sample_submission.csv')
predictions = model.predict(test_x.astype('float32'))
binary_predictions = (predictions >= 0.5).astype(int)
sample_submission["target"] = binary_predictions
sample_submission.to_csv("../data/submission.csv", index=False)

313/313 [==============================] - 0s 789us/step


In [291]:
submit.to_csv("../submission_data/5.csv", index = False)

In [46]:
pred_final = model_xgb.predict(test)

In [47]:
submit = pd.DataFrame({
    'user_id': test_id,
    'target': pred_final.flatten()
})

In [48]:
submit.to_csv("../submission_data/5.csv", index = False)